# Flujo completo de un modelo de RAG


In [1]:
import os
from typing import List
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.documents import Document
import re
from typing import List, Dict
from langchain.document_loaders import PyPDFLoader

## Script Limpieza y Segmentación

In [5]:
# Función para cargar documentos PDF
def load_pdf_documents(file_path: str) -> List[str]:
    """
    Carga documentos PDF y devuelve una lista de páginas como texto.

    Args:
        file_path (str): Ruta del archivo PDF a cargar.

    Returns:
        List[str]: Lista de textos extraídos de cada página del PDF.
    """
    loader = PyPDFLoader(file_path)
    docs = loader.load()
    return [doc.page_content for doc in docs]  # Extrae texto como lista de strings


# Función para limpiar el texto y excluir secciones específicas
def clean_text_and_exclude_sections(text: str) -> str:
    """
    Limpia el texto extraído de un PDF, excluyendo índices y bibliografías 
    basados en patrones comunes.

    Args:
        text (str): Texto extraído del PDF.

    Returns:
        str: Texto limpio y sin secciones de índice o bibliografía.
    """
    # Reemplazar saltos de línea por espacios
    text = re.sub(r'\n+', ' ', text)
    
    # Eliminar espacios múltiples
    text = re.sub(r'\s+', ' ', text)
    
    # Excluir secciones específicas (Índice, Referencias, Bibliografía)
    text = re.sub(r'(?i)(Índice|Table of Contents).*?(Referencias|Bibliografía|References)', '', text, flags=re.DOTALL)
    
    # Opcional: Detectar bibliografías al final del documento
    text = re.sub(r'(?i)(Referencias|Bibliografía|References).*$', '', text, flags=re.DOTALL)
    
    # Eliminar caracteres no deseados (mantener alfanuméricos y signos de puntuación básicos)
    text = re.sub(r'[^a-zA-Z0-9.,;?!:()\s]', '', text)
    
    # Corrección de palabras divididas por guiones al final de línea (ej., "pro-\nject" -> "project")
    text = re.sub(r'-\s', '', text)
    
    # Eliminar espacios al inicio y final
    text = text.strip()
    
    return text


# Función para dividir texto en oraciones
def split_text_into_sentences(text: str) -> List[Dict[str, str]]:
    """
    Divide un texto en oraciones basado en '.', '?', y '!' y devuelve una lista de diccionarios.

    Args:
        text (str): El texto a dividir.

    Returns:
        List[Dict[str, str]]: Lista de diccionarios con 'sentence' y 'index'.
    """
    single_sentences_list = re.split(r'(?<=[.?!])\s+', text.strip())
    sentences = [{'sentence': sentence, 'index': i} for i, sentence in enumerate(single_sentences_list)]
    return sentences


# Función para combinar oraciones
def combine_sentences(sentences: List[Dict[str, str]], buffer_size: int = 1) -> List[Dict[str, str]]:
    """
    Combina oraciones de acuerdo al tamaño del buffer definido.

    Args:
        sentences (List[Dict[str, str]]): Lista de oraciones con índices.
        buffer_size (int): Número de oraciones antes y después a combinar.

    Returns:
        List[Dict[str, str]]: Lista con oraciones combinadas.
    """
    for i in range(len(sentences)):
        combined_sentence = ''

        # Añadir oraciones previas
        for j in range(i - buffer_size, i):
            if j >= 0:
                combined_sentence += sentences[j]['sentence'] + ' '

        # Añadir oración actual
        combined_sentence += sentences[i]['sentence']

        # Añadir oraciones posteriores
        for j in range(i + 1, i + 1 + buffer_size):
            if j < len(sentences):
                combined_sentence += ' ' + sentences[j]['sentence']

        # Guardar la oración combinada en el dict actual
        sentences[i]['combined_sentence'] = combined_sentence.strip()

    return sentences

file_path = "../practicos-rag/data/USA/CFR-2024-vol1.pdf"
buffer_size = 2  # Número de oraciones antes y después a combinar
# Cargar texto del PDF
pdf_docs = load_pdf_documents(file_path)

# Combinar texto de todas las páginas
full_text = " ".join(pdf_docs)

# Limpiar texto
cleaned_text = clean_text_and_exclude_sections(full_text)

# Dividir en oraciones
sentences = split_text_into_sentences(cleaned_text)

# Combinar oraciones
combined_sentences = combine_sentences(sentences, buffer_size)



In [6]:
documents_for_qdrant = [doc["combined_sentence"] for doc in combined_sentences]

# Embeddings

In [8]:
from langchain_qdrant import QdrantVectorStore, RetrievalMode
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.schema import Document

# Crear embeddings
open_source_embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-MiniLM-L6-v2")

# Preparar documentos para Qdrant
documents_for_qdrant = [
    Document(page_content=doc["combined_sentence"], metadata={"index": doc["index"]})
    for doc in combined_sentences
]

# Crear la tienda de vectores
qdrant = QdrantVectorStore.from_documents(
    documents_for_qdrant,
    embedding=open_source_embeddings,
    location=":memory:",
    collection_name="my_documents",
    retrieval_mode=RetrievalMode.DENSE,
)

# Consulta
query = "What did the president say about Ketanji Brown Jackson?"
found_docs = qdrant.similarity_search(query)

# Imprimir resultados
for doc in found_docs:
    print(f"Content: {doc.page_content}, Metadata: {doc.metadata}")


Content: 1.382 What labeling or marking require ments apply to a detained article of food? 1.383 What expedited procedures apply when FDA initiates a seizure action against a detained perishable food? 1.384 When does a detention order termi nate? HOW DOES FDA ORDER A DETENTION? 1.391 Who approves a detention order?, Metadata: {'index': 75, '_id': '918e0990c5be4c568a17ae195a4e4d02', '_collection_name': 'my_documents'}
Content: 1.392 Who receives a copy of the detention order? 1.393 What information must FDA include in the detention order? WHAT IS THE APPEAL PROCESS FOR A DETENTION ORDER? 1.401 Who is entitled to appeal? 1.402 What are the requirements for submit ting an appeal?, Metadata: {'index': 80, '_id': 'd5e4d42ed45448a3be827b4765b3c6e5', '_collection_name': 'my_documents'}
Content: 1.380 Where and under what conditions must the detained article of food be held? 1.381 May a detained article of food be de livered to another entity or transferred to another location? 1.382 What labe